In [ ]:
import pandas as pd
import chardet as cd
from pandas import DatetimeTZDtype as dtime

# Use detect() to detect the encoding of the file
with open("HW1data.csv", 'rb') as testdata:
    result = cd.detect(testdata.read(10000))
coding = result['encoding']
# Read the csv file
H_data = pd.read_csv("HW1data.csv", encoding=coding)
H_data


In [ ]:
# 2 Data clean
# (a) Deal with Patient with 62299 rows
PatientGroup_dict = H_data.groupby('PatientId').agg({
    'Gender': 'first',
    'Age': 'first',
    'Neighbourhood': 'first',
}).to_dict()
#PatientGroup
# (b) Drop na of PatientId, ScheduledDay, AppointmentDay, SMS_received, No-show
H_data_clean = H_data.dropna(subset=['PatientId', 'ScheduledDay', 'AppointmentDay', 'SMS_received', 'No-show']).copy()
# (c) Fill na of Gender, Age, Neighbourhood with PatientGroup
H_data_clean.loc[:, 'Age'] = H_data_clean['Age'].fillna(H_data_clean['PatientId'].map(PatientGroup_dict['Age']))
H_data_clean.loc[:, 'Gender'] = H_data_clean['Gender'].fillna(H_data_clean['PatientId'].map(PatientGroup_dict['Gender']))
H_data_clean.loc[:, 'Neighbourhood'] = H_data_clean['Neighbourhood'].fillna(H_data_clean['PatientId'].map(PatientGroup_dict['Neighbourhood']))

H_data_clean


In [ ]:
# 4 Fill the missing values of Schloarship, Hipertension, Diabetes, Alcoholism, Handcap with 0
H_data_fill = H_data_clean.copy()
H_data_fill.loc[:, 'Scholarship'] = H_data_fill['Scholarship'].fillna(0)
H_data_fill.loc[:, 'Hipertension'] = H_data_fill['Hipertension'].fillna(0)
H_data_fill.loc[:, 'Diabetes'] = H_data_fill['Diabetes'].fillna(0)
H_data_fill.loc[:, 'Alcoholism'] = H_data_fill['Alcoholism'].fillna(0)
H_data_fill.loc[:, 'Handcap'] = H_data_fill['Handcap'].fillna(0)
H_data_fill

In [ ]:
# 5 delete two column PatientId and AppointmentID 
H_data_deleteCol = H_data_fill.drop(columns=['PatientId', 'AppointmentID']).copy()
H_data_deleteCol

In [ ]:
# 6 remove the row with whose age has problem
# According to science a person can be with age between 0 and 150
H_data_AgeProc = H_data_deleteCol[(H_data_deleteCol['Age'] >= 0) & (H_data_deleteCol['Age'] <= 150)].copy()
H_data_AgeProc

In [ ]:
# 7 Add a column name Delta_Day which is the substraction between ScheduledDay and AppointmentDay
H_data_DeltaDay = H_data_AgeProc.copy()
# Change the dtype of ScheduledDay and AppointmentDay so that they can calculate
H_data_DeltaDay.loc[:, 'ScheduledDay'] = pd.to_datetime(H_data_DeltaDay['ScheduledDay'], utc=True)
H_data_DeltaDay.loc[:, 'AppointmentDay'] = pd.to_datetime(H_data_DeltaDay['AppointmentDay'], utc=True)
# Add a column named Delta_Day (only add the day)
H_data_DeltaDay['Delta_Day'] = (H_data_DeltaDay['AppointmentDay']-H_data_DeltaDay['ScheduledDay']).dt.days
H_data_DeltaDay

In [ ]:
# 8 Add SDay_DOW and ADay_DOW (day of week for ScheduledDay and AppointmentDay)
H_data_weekday = H_data_DeltaDay.copy()
# A dict for day_of_week to its names
weekday_dict = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
# Add SDay and ADay accordint to SD and AD's dt.day_of_week
H_data_weekday['SDay_DOW'] = H_data_weekday['ScheduledDay'].dt.day_of_week.map(weekday_dict)
H_data_weekday['ADay_DOW'] = H_data_weekday['AppointmentDay'].dt.day_of_week.map(weekday_dict)
H_data_weekday

In [ ]:
# 9 delete ScheduledDay and AppointmentDay two columns
H_data_final = H_data_weekday.drop(columns=['ScheduledDay', 'AppointmentDay']).copy()
H_data_final